### 0. KIS OpenAPI 토큰 발급

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

In [2]:
#홈페이지에서 API서비스 신청시 받은 Appkey, Appsecret 값 설정
#모의투자
#my_app: "앱키"
#my_sec: "앱시크리트"
#실전투자 
load_dotenv()

my_app= os.getenv("MY_APP")
my_sec= os.getenv("MY_SEC")

#계좌번호 앞 8자리
my_acct="43016876"
my_acct_stock= "증권계좌 8자리"
my_acct_future= "선물옵션계좌 8자리"
#계좌번호 뒤 2자리
my_prod= "01"
#my_prod: "03"

#실전투자
prod= "https://openapi.koreainvestment.com:9443"
#모의투자
vps= "https://openapivts.koreainvestment.com:29443"


my_agent= "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

class Env:
    def __init__(self):
        self.my_app = my_app
        self.my_sec = my_sec
        self.my_url = prod
        self.my_token = None
        self.my_acct = my_acct
        self.my_prod = my_prod
_env = Env()

In [3]:
def auth():
    url = f'{_env.my_url}/oauth2/tokenP'
    headers = {"content": "application/json"}
    data = {
        "grant_type":"client_credentials",
        "appkey": _env.my_app,
        "appsecret": _env.my_sec
    }

    res = requests.post(url, headers=headers, data= json.dumps(data))
    if res.status_code == 200:
        print('토큰 발급 성공')
        _env.my_token = res.json()['access_token']
    else:
        print("토큰 발급 실패")
        print(res.status_code, res.text)

In [4]:
auth()

토큰 발급 성공


### 1. 상장된 모든 회사 종목 조회

In [15]:
from pykrx import stock

In [16]:
today = pd.Timestamp("today").strftime("%Y%m%d")


In [17]:
# KOSPI 전체 종목
kospi = stock.get_market_ticker_list(today, market="KOSPI")
kospi_df = pd.DataFrame([(ticker, stock.get_market_ticker_name(ticker), "KOSPI") for ticker in kospi],
                        columns=["종목코드", "종목명", "시장"])

# KOSDAQ 전체 종목
kosdaq = stock.get_market_ticker_list(today, market="KOSDAQ")
kosdaq_df = pd.DataFrame([(ticker, stock.get_market_ticker_name(ticker), "KOSDAQ") for ticker in kosdaq],
                         columns=["종목코드", "종목명", "시장"])

In [18]:
tickers_df = pd.concat([kospi_df, kosdaq_df], ignore_index=True)

In [19]:
tickers_df

,종목코드,종목명,시장
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI
...,...,...,...
2752,024060,흥구석유,KOSDAQ
2753,010240,흥국,KOSDAQ
2754,189980,흥국에프엔비,KOSDAQ
2755,037440,희림,KOSDAQ


### 1. KIS OpenAPI 호출 예시
- 삼성전자(005930) 현재가 데이터 조회

In [ ]:
stock_code = "005930"
market_code = 'J'
url = f'{_env.my_url}/uapi/domestic-stock/v1/quotations/inquire-price'

In [32]:
headers = {
    "Content-Type": "application/json",
    "authorization": f'Bearer {_env.my_token}',
    "appkey": _env.my_app,
    "appsecret": _env.my_sec,
    "tr_id": "FHKST01010100",  # 국내주식 현재가 시세 조회
    "custtype": "P"
}

In [ ]:
params = {
    "fid_cond_mrkt_div_code": market_code,  
    "fid_input_iscd": stock_code
}

In [55]:
response = requests.get(url, headers=headers, params=params)

In [56]:
if response.status_code != 200:
    print('조회 실패')
else:
    print('조회 성공')
    data = response.json()
    df = pd.DataFrame([data["output"]])
    display(df)

조회 성공


,iscd_stat_cls_code,marg_rate,rprs_mrkt_kor_name,bstp_kor_isnm,temp_stop_yn,oprc_rang_cont_yn,clpr_rang_cont_yn,crdt_able_yn,grmn_rate_cls_code,elw_pblc_yn,...,cpfn_cnnm,frgn_hldn_qty,vi_cls_code,ovtm_vi_cls_code,last_ssts_cntg_qty,invt_caful_yn,mrkt_warn_cls_code,short_over_yn,sltr_yn,mang_issu_cls_code
0,55,20.00,KOSPI200,전기·전자,N,N,N,Y,40,Y,...,"7,780 억",2963359261,N,N,175324,N,00,N,N,N


### 2. 상장된 모든 회사에 대한 고정 정보 조회
- 종목 코드
- 대표 시장 명
- 업종명
- 자본금
- 상장 주식 수
- 결산월
- 데이터 입력 시각
- 데이터 수정 시각

In [44]:
import time

In [68]:
def get_company_static_info(stock_code, market):
    url = f"{_env.my_url}/uapi/domestic-stock/v1/quotations/inquire-price"
    params = {
        "fid_cond_mrkt_div_code": "J" if market == "KOSPI" or market == "KOSDAQ" else "NX",  # 9로 시작하면 코스닥
        "fid_input_iscd": stock_code
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if response.status_code == 200:
        out = data["output"]
        print(out)
        return {
            "종목 코드": out["stck_shrn_iscd"],
            "대표 시장 명": out["rprs_mrkt_kor_name"],
            "industry": out.get("bstp_kor_isnm", ""),
            "자본금(억 원)": out["cpfn"],
            "상장 주식 수": out["lstn_stcn"],
            "결산월": out["stac_month"],
            "created_at": datetime.now(),
            "updated_at": datetime.now()
        }
    else:
        print('조회 실패')
        return None

In [58]:
results = []

for i, row in tickers_df.iterrows():  
    code = row['종목코드']
    name = row['종목명']
    
    # print(f"[{i+1}] 종목코드: {code}, 종목명: {name}")
    
    info = get_company_static_info(code)
    print(info)
    if info:
        info['종목명'] = name
        results.append(info)
    
    time.sleep(0.3)  # API 호출 간격 제한

df_companies = pd.DataFrame(results)
display(df_companies)

{'종목 코드': '095570', '대표 시장 명': 'KOSPI', '업종명': '일반서비스', '자본금(억 원)': '468', '상장 주식 수': '45252759', '결산월': '12', 'created_at': datetime.datetime(2025, 4, 18, 1, 22, 14, 728499), 'updated_at': datetime.datetime(2025, 4, 18, 1, 22, 14, 728499)}
{'종목 코드': '006840', '대표 시장 명': 'KOSPI', '업종명': '금융', '자본금(억 원)': '662', '상장 주식 수': '13247561', '결산월': '12', 'created_at': datetime.datetime(2025, 4, 18, 1, 22, 15, 166917), 'updated_at': datetime.datetime(2025, 4, 18, 1, 22, 15, 166917)}
{'종목 코드': '027410', '대표 시장 명': 'KOSPI', '업종명': '금융', '자본금(억 원)': '957', '상장 주식 수': '95716791', '결산월': '12', 'created_at': datetime.datetime(2025, 4, 18, 1, 22, 15, 543822), 'updated_at': datetime.datetime(2025, 4, 18, 1, 22, 15, 543822)}
{'종목 코드': '282330', '대표 시장 명': 'KOSPI200', '업종명': '유통', '자본금(억 원)': '173', '상장 주식 수': '17283906', '결산월': '12', 'created_at': datetime.datetime(2025, 4, 18, 1, 22, 15, 959694), 'updated_at': datetime.datetime(2025, 4, 18, 1, 22, 15, 959694)}
{'종목 코드': '138930', '대표 시장 명': 'KOSPI200',

KeyboardInterrupt: 

In [70]:
get_company_static_info("278990", "KOSDAQ")

{'iscd_stat_cls_code': '57', 'marg_rate': '100.00', 'rprs_mrkt_kor_name': 'KONEX', 'temp_stop_yn': 'N', 'oprc_rang_cont_yn': 'N', 'clpr_rang_cont_yn': 'N', 'crdt_able_yn': 'N', 'grmn_rate_cls_code': ' ', 'elw_pblc_yn': 'N', 'stck_prpr': '3800', 'prdy_vrss': '100', 'prdy_vrss_sign': '2', 'prdy_ctrt': '2.70', 'acml_tr_pbmn': '3800', 'acml_vol': '1', 'prdy_vrss_vol_rate': '0.16', 'stck_oprc': '3800', 'stck_hgpr': '3800', 'stck_lwpr': '3800', 'stck_mxpr': '4255', 'stck_llam': '3145', 'stck_sdpr': '3700', 'wghn_avrg_stck_prc': '3800.00', 'hts_frgn_ehrt': '0.00', 'frgn_ntby_qty': '0', 'pgtr_ntby_qty': '0', 'pvt_scnd_dmrs_prc': '3900', 'pvt_frst_dmrs_prc': '3800', 'pvt_pont_val': '3600', 'pvt_frst_dmsp_prc': '3500', 'pvt_scnd_dmsp_prc': '3300', 'dmrs_val': '3850', 'dmsp_val': '3550', 'cpfn': '24', 'rstc_wdth_prc': '555', 'stck_fcam': '500', 'stck_sspr': '2290', 'aspr_unit': '5', 'hts_deal_qty_unit_val': '1', 'lstn_stcn': '4836831', 'hts_avls': '184', 'per': '7.24', 'pbr': '0.62', 'stac_month'

{'종목 코드': '278990',
 '대표 시장 명': 'KONEX',
 'industry': '',
 '자본금(억 원)': '24',
 '상장 주식 수': '4836831',
 '결산월': '12',
 'created_at': datetime.datetime(2025, 4, 18, 2, 33, 22, 660914),
 'updated_at': datetime.datetime(2025, 4, 18, 2, 33, 22, 660914)}